In [1]:
import yt
import numpy as np
import os
import pandas as pd

In [4]:
curr_dir = os.getcwd()
all_files = os.listdir(curr_dir+"/mu_-0.2/")
for filename in all_files:
    if filename.startswith("inputs"):
        input_file = filename
input_file = curr_dir+"/mu_-0.2/"+input_file
fo = open(input_file, "r")

In [5]:
def extract_parameters(fo):
    parameters = dict()
    for line in fo.readlines():
        if line.startswith("nsteps"):
            temp = line.split()
            parameters["nL"] = int(temp[-1])
        elif line.startswith("n_cell"):
            temp = line.split()
            parameters["Nx"] = int(temp[2])
            parameters["Nt"] = int(temp[-1])
        elif line.startswith("nrrb.l"):
            temp = line.split()
            parameters["lambda"] = float(temp[-1])
        elif line.startswith("nrrb.mu"):
            temp = line.split()
            parameters["mu"] = float(temp[-1])
        elif line.startswith("nrrb.m"):
            temp = line.split()
            parameters["m"] = float(temp[-1])
        elif line.startswith("nrrb.w"):
            temp = line.split()
            parameters["wz"] = float(temp[-1])
        elif line.startswith("nrrb.w_t"):
            temp = line.split()
            parameters["wtr"] = float(temp[-1])
        elif line.startswith("nrrb.dtau"):
            temp = line.split()
            parameters["dt"] = float(temp[-1])
        elif line.startswith("nrrb.eps"):
            temp = line.split()
            parameters["eps"] = float(temp[-1])
    parameters["beta"] = parameters["dt"]*parameters["Nt"]
    return parameters

parameters = extract_parameters(fo)
print(parameters)

{'m': 1.0, 'lambda': 0.0, 'wz': 0.0, 'dt': 0.05, 'mu': -0.2, 'eps': 0.001, 'Nx': 21, 'Nt': 80, 'nL': 100000, 'beta': 4.0}


In [6]:
nL = parameters['nL']
all_steps = np.arange(1,nL+1,1)
all_plotfiles = []
for pltfile in all_files:
    if pltfile.startswith("plt"):
        all_plotfiles.append(int(pltfile[3:]))
#print(all_plotfiles)

In [7]:
def find_missing_files(all_steps, all_plotfiles):
    missing = []
    for step in all_steps:
        if step not in all_plotfiles:
            missing.append(step)
    return missing

In [8]:
#force nL to be smaller
#nL = len(all_plotfiles)
nL = 20
missing = find_missing_files(np.arange(1,nL+1), all_plotfiles)
print(len(missing))

11


In [17]:
class GridData(object):
    def __init__(self, ds):
        # store data arrays for the dataset
        cg = ds.covering_grid(left_edge=ds.domain_left_edge, dims=ds.domain_dimensions, level=0)
        coords = cg.fcoords

        xs = coords[:,0]
        ys = coords[:,1]
        ts = coords[:,2]

        self.dx = np.max(xs[1:] - xs[:-1])
        self.dy = np.max(ys[1:] - ys[:-1])
        self.dt = np.max(ts[1:] - ts[:-1])

        self.phi_1_Re = cg["phi_1_Re"][:,:,:].d
        self.phi_1_Im = cg["phi_1_Im"][:,:,:].d
        self.phi_2_Re = cg["phi_2_Re"][:,:,:].d
        self.phi_2_Im = cg["phi_2_Im"][:,:,:].d

    def get_fields(self,i,j,t):
        return complex(self.phi_1_Re[i,j,t], self.phi_1_Im[i,j,t]), complex(self.phi_2_Re[i,j,t], self.phi_2_Im[i,j,t])

In [18]:
work_dir = curr_dir+"/mu_-0.2/"
filename = "plt"+str(all_plotfiles[0]).zfill(5) 
ds = yt.load(work_dir+filename)
grid_data = GridData(ds)

yt : [INFO     ] 2020-01-14 10:25:45,796 Parameters: current_time              = 0.009000000000000001
yt : [INFO     ] 2020-01-14 10:25:45,797 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:25:45,798 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:25:45,799 Parameters: domain_right_edge         = [21. 21. 80.]


In [19]:
grid_data.get_fields(0,0,0)

((-1.0193617570850662+0.7897918574886916j),
 (0.5370442239338948+0.2010187190034365j))

Average density: $\langle \hat{n} \rangle = \frac{1}{N_{x}^{d}}\sum_{x}n_{x}$

where $n_{x}$ is the local density: $n_{x}= \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}-i \phi_{2,x})(\phi_{1,x-\hat{\tau}}+i \phi_{2,x-\hat{\tau}})$

Using our four complexified fields:
\begin{eqnarray}
n_{x} &=& \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}^{R} + i \phi_{1,x}^{I}-i \phi_{2,x}^{R} +\phi_{2,x}^{I})(\phi_{1,x-\hat{\tau}}^{R} + i \phi_{1,x-\hat{\tau}}^{I} +i \phi_{2,x-\hat{\tau}}^{R} - \phi_{2,x-\hat{\tau}}^{I})\\
&=& \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}^{R} + i \phi_{1,x}^{I}-i \phi_{2,x}^{R} +\phi_{2,x}^{I})(\phi_{1,x-\hat{\tau}}^{R} + i \phi_{1,x-\hat{\tau}}^{I} +i \phi_{2,x-\hat{\tau}}^{R} - \phi_{2,x-\hat{\tau}}^{I})
\end{eqnarray}

Question for self: do you need to average over all t?

Figure out how to deal with boundary conditions

In [31]:
def local_density(gd, i, j, Nt, beta, mu):
    n = complex(0.,0.)
    for t in np.arange(1,Nt):
        phi1,phi2 = gd.get_fields(i,j,t)
        phi1t,phi2t = gd.get_fields(i,j,t-1)
        n += complex(phi1, -phi2)*complex(phi1t, -phi2t)
    phi1,phi2 = gd.get_fields(i,j,0)
    phi1t,phi2t = gd.get_fields(i,j,Nt-1)
    n += complex(phi1, -phi2)*complex(phi1t, -phi2t)
    return 0.5*np.exp(-beta*mu/float(Nt))*n/float(Nt)

def total_density(gd, Nx, Nt, beta, mu):
    ntot = complex(0.,0.)
    volume = Nx**2
    for i in np.arange(0,Nx):
        for j in np.arange(0,Nx):
            ntot += local_density(gd, i, j, Nt, beta, mu)
    return ntot/volume

In [32]:
nL = 9
Nx = parameters['Nx']
Nt = parameters['Nt']
beta = parameters['beta']
mu = parameters['mu']
df = pd.DataFrame(columns=['step','density'])
df.head()
work_dir = curr_dir+"/mu_-0.2/"
step_arr = []
dens_arr = []
for step in np.arange(1,nL+1):
    filename = "plt"+str(step).zfill(5) 
    ds = yt.load(work_dir+filename)
    gd = GridData(ds)
    N = total_density(gd, Nx, Nt, beta, mu)
    df.append({'step': step, 'density': N}, ignore_index=True)
    step_arr.append(step)
    dens_arr.append(N)
df.head()

for s, N in zip(step_arr, dens_arr):
    print("{}: {}".format(s,N))

yt : [INFO     ] 2020-01-14 10:35:24,581 Parameters: current_time              = 0.001
yt : [INFO     ] 2020-01-14 10:35:24,583 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:35:24,584 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:35:24,587 Parameters: domain_right_edge         = [21. 21. 80.]
yt : [INFO     ] 2020-01-14 10:35:25,454 Parameters: current_time              = 0.002
yt : [INFO     ] 2020-01-14 10:35:25,456 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:35:25,458 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:35:25,460 Parameters: domain_right_edge         = [21. 21. 80.]


plt00001


yt : [INFO     ] 2020-01-14 10:35:26,349 Parameters: current_time              = 0.003
yt : [INFO     ] 2020-01-14 10:35:26,350 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:35:26,351 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:35:26,353 Parameters: domain_right_edge         = [21. 21. 80.]


plt00002


yt : [INFO     ] 2020-01-14 10:35:27,292 Parameters: current_time              = 0.004
yt : [INFO     ] 2020-01-14 10:35:27,293 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:35:27,295 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:35:27,297 Parameters: domain_right_edge         = [21. 21. 80.]


...

plt00006


yt : [INFO     ] 2020-01-14 10:35:32,024 Parameters: current_time              = 0.008
yt : [INFO     ] 2020-01-14 10:35:32,025 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:35:32,026 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:35:32,027 Parameters: domain_right_edge         = [21. 21. 80.]


plt00007


yt : [INFO     ] 2020-01-14 10:35:33,068 Parameters: current_time              = 0.009000000000000001
yt : [INFO     ] 2020-01-14 10:35:33,069 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:35:33,073 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:35:33,075 Parameters: domain_right_edge         = [21. 21. 80.]


plt00008
plt00009
1: (-0.0026385907472054227+0.002099770830271272j)
2: (-0.002399761033836866+0.001983534949230471j)
3: (-0.0022511918084475216+0.0015490847786933822j)
4: (-0.0021298563493369143+0.0012472469123865758j)
5: (-0.0021840063305969574+0.0012140739124539485j)
6: (-0.002091769200698329+0.0011174625792657994j)
7: (-0.0020711114321221363+0.0009911011044013486j)
8: (-0.0018280635483297713+0.0011607958300482659j)
9: (-0.001953305400446652+0.0011677016043066847j)
